Documentation on h5py [here](https://docs.h5py.org/en/stable/quick.html)

In [ ]:
import h5py
import numpy as np
from os import path

In [ ]:
fname = "test_h5py.h5"

Writing to a file

In [ ]:
with h5py.File(fname, "w") as fh:
    dset = fh.create_dataset("mydataset", (100,), dtype='i')

Reading from a file

In [ ]:
with h5py.File(fname, "r") as fh:
    print(fh["mydataset"][:])

Reading data written by R rhdf5 package (if it exists)

In [ ]:
fname = "test_rhdf5.h5"
if path.isfile(fname):
    with h5py.File(fname) as fh:
        print(fh.keys())
        print(fh["foo"].keys())
        print(fh["foo/A"][:])
        print(fh["df"][:])

Reading data written by C++ HighFive package (if it exists)

In [ ]:
fname = "test_highfive.h5"
if path.isfile(fname):
    with h5py.File(fname) as fh:
        print(fh["path/to/A"][()])